In [1]:
# define the calling function
run_method<- function(raw_data, DA_method,run_id){

  if(DA_method=="ANCOM-BC"){
    out<- run_ancombc(raw_data = raw_data,run_id=run_id)
  } else if(DA_method=="FlashWeave"){
    out<- run_flashweave(raw_data = raw_data,run_id=run_id)
  
  } else if(DA_method=="ANCOM-BC2"){
     out<- run_ancombc2(raw_data = raw_data) 
  }

  return(out)
}

In [17]:
run_ancombc<- function(raw_data,run_id=run_id){

    
    # data file paths
    simulated_data<-"/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/otu_table1.txt"
    biom_file <- '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/otu_table1.biom'
    output_qza <- '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/otu_table1.qza'
    metadata<-"/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/meta_table1.tsv"
    ancomoutput <- '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/ancombc.qza'
    exported <- '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/exported_ancompd'
    ppd_otu_filtered_qza="/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/otu_pd_qiime_filtered.qza"
    base_path <- "/home/zakir/from_pendrive/PhD/deep_learning/graph_nn/DAA_using_GNN"
    exporteddd <- paste0( base_path,"/exported_net_" ,run_id )
    
    
    meta.dat <- data.frame(X = raw_data$covariate, Z1 = raw_data$confounder[, 1],
                       Z2 = raw_data$confounder[, 2])
    otu.tab.sim <- raw_data$otu.tab.sim
    actual=ifelse(raw_data$diff.otu.ind, "DA", "NOT_DA")
    tr=data.frame("truth" = actual, row.names = rownames(otu.tab.sim), stringsAsFactors = F)
    meta.dat$X=ifelse(meta.dat$X, "healthy","unhealthy")
    #otu data processing for qiime2 input
    otu <- cbind(rownames(otu.tab.sim), otu.tab.sim[,2:length(colnames(otu.tab.sim))])
    colnames(otu)[1] <- "featureid"
    rownames(otu) <- NULL

    write.table(otu, simulated_data, sep = "\t", col.names=TRUE, row.names=FALSE, quote = FALSE)
    #metadata processing for qiime2 input
    colnames(meta.dat)=c("covariate", "cofounder1","cofounder2")

    metad  <- cbind(rownames(meta.dat),meta.dat)
    colnames(metad)[1] <- "sampleid"
    rownames(metad) <- NULL
    write.table(metad, metadata, sep = "\t", col.names=TRUE, row.names=FALSE, quote = FALSE)

   # convert data into biom format
    system(paste("biom convert",
                 "-i",simulated_data ,
                 "-o", biom_file ,
                 "--table-type=\"OTU table\"",
                 "--to-hdf5"), intern=TRUE)
    #import data into qiime2                
    system(paste("qiime tools import",
             "--input-path", biom_file,
             "--type 'FeatureTable[Frequency]'",
             "--input-format BIOMV210Format",
             "--output-path", output_qza), intern=TRUE)
    
    #ppd_otu_filtered_qza="~/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/data/PD/otu_pd_qiime_filtered.qza"
    system(paste("qiime feature-table filter-features",
             "--i-table", output_qza,
             "--p-min-frequency",50,
             "--p-min-samples",4,
             "--o-filtered-table", ppd_otu_filtered_qza), intern=TRUE)
                   
   # Run ANCOM-BC with sampled parameters
    system(paste("qiime composition ancombc",
             "--i-table",ppd_otu_filtered_qza,
             "--m-metadata-file", metadata,
             "--p-formula", "covariate",
             "--p-p-adj-method", "BH",
             "--p-neg-lb", FALSE,
             "--p-tol", 0.01,
             "--p-max-iter", 75,
             "--p-conserve", FALSE,
             "--o-differentials", ancomoutput))                
  
   # Export the results from QIIME2
   system(paste("qiime tools export", 
              "--input-path",ancomoutput,
             "--output-path",exported))                
  
  # Construct file path for q_val_slice.csv
   q_val_csv <- paste0(exported, "/q_val_slice.csv")
  
  # Check if the file exists before reading
  if (!file.exists(q_val_csv)) {
    cat("Expected q_val_slice.csv does not exist at path:", q_val_csv, "\n")
    next
  }
  
    # Read the q-value results
    ancom_q_val <- read.csv(file = q_val_csv, sep = ',', row.names = 1)

    # take only necessary data
    ancom_q_val=data.frame("covariateunhealthy" = ancom_q_val$covariateunhealthy, row.names = rownames(ancom_q_val), stringsAsFactors = F)
    #construct output dataframe
    e=!(rownames(otu.tab.sim)%in%rownames(ancom_q_val))
    nq=rownames(otu.tab.sim)[e]
    nqv=data.frame("covariateunhealthy" = rep(NA, length(nq)), row.names = nq, stringsAsFactors = F)
    q_val_all=rbind(ancom_q_val,nqv)
    names(q_val_all)[names(q_val_all) == "covariateunhealthy"] <- "q_value"
    pred= ifelse(ancom_q_val < 0.05, "DA", "NOT_DA")
    pred=as.data.frame(pred)
    d=!(rownames(otu.tab.sim)%in%rownames(pred))
    npred=rownames(otu.tab.sim)[d]
    ND=data.frame("covariateunhealthy" = rep("NOT_DA", length(npred)), row.names = npred, stringsAsFactors = F)
    pr=rbind(pred,ND)
    out_df=cbind(pr,q_val_all,tr)
    names(out_df)[names(out_df) == "covariateunhealthy"] <- "pred"

  return("method_out"=out_df)
}


In [3]:
base_path <- "/home/zakir/from_pendrive/PhD/deep_learning/graph_nn/DAA_using_GNN"
run_id="sample1"
paste0( base_path,"/exported_net_" ,run_id )

[1] "/home/zakir/from_pendrive/PhD/deep_learning/graph_nn/DAA_using_GNN/exported_net_sample1"

In [4]:
run_flashweave<- function(raw_data,run_id){

    # data file paths 
    simulated_data<-"/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/otu_table1.txt"
    biom_file <- '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/otu_table1.biom'
    output_qza <- '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/otu_table1.qza'
    metadata<-"/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/meta_table1.tsv"
    output_flash <- '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/flashnet1.qza'
    ppd_otu_filtered_qza="/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/otu_pd_qiime_filtered.qza"
    base_path <- "/home/zakir/from_pendrive/PhD/deep_learning/graph_nn/DAA_using_GNN"
    exported <- paste0( base_path,"/exported_net_" ,run_id )
    meta.dat <- data.frame(X = raw_data$covariate, Z1 = raw_data$confounder[, 1],
                       Z2 = raw_data$confounder[, 2])
    otu.tab.sim <- raw_data$otu.tab.sim
    actual=ifelse(raw_data$diff.otu.ind, "DA", "NOT_DA")
    tr=data.frame("truth" = actual, row.names = rownames(otu.tab.sim), stringsAsFactors = F)
     meta.dat$X=ifelse(meta.dat$X, "healthy","unhealthy")
    #otu data processing for qiime2 input
    otu <- cbind(rownames(otu.tab.sim), otu.tab.sim[,2:length(colnames(otu.tab.sim))])
    colnames(otu)[1] <- "featureid"
    rownames(otu) <- NULL

    write.table(otu, simulated_data, sep = "\t", col.names=TRUE, row.names=FALSE, quote = FALSE)
    #metadata processing for qiime2 input
    colnames(meta.dat)=c("covariate", "cofounder1","cofounder2")

    metad  <- cbind(rownames(meta.dat),meta.dat)
    colnames(metad)[1] <- "sampleid"
    rownames(metad) <- NULL
    write.table(metad, metadata, sep = "\t", col.names=TRUE, row.names=FALSE, quote = FALSE)

   # convert data into biom format
    system(paste("biom convert",
                 "-i",simulated_data ,
                 "-o", biom_file ,
                 "--table-type=\"OTU table\"",
                 "--to-hdf5"), intern=TRUE)

    #import data into qiime 
    system(paste("qiime tools import",
             "--input-path", biom_file,
             "--type 'FeatureTable[Frequency]'",
             "--input-format BIOMV210Format",
             "--output-path", output_qza), intern=TRUE)
    
    # filtering data
    # if want filtering then skip this step
    system(paste("qiime feature-table filter-features",
             "--i-table", output_qza,
             "--p-min-frequency",50,
             "--p-min-samples",4,
             "--o-filtered-table", ppd_otu_filtered_qza), intern=TRUE)
  
    # Run FlashWeave with sampled parameters
    system(paste("qiime makarsa flashweave",
             "--i-table", ppd_otu_filtered_qza,
             "--m-metadata-file",metadata,
             "--p-heterogeneous", TRUE,         
             "--p-sensitive", TRUE,             
             "--p-max-k", 0,
             "--p-alpha", 0.04 ,
             "--o-network", output_flash), intern=TRUE)                              
                  

   # Export the results from QIIME2
   system(paste("qiime tools export", 
              "--input-path",output_flash,
             "--output-path",exported), intern=TRUE)                
  
  
  
  # Construct file path for metadata.tsv
   exported_net <- paste0(exported, "/network.graphml")
    
  return("method_out"=exported_net)
}

In [ ]:
ancombc<- function(raw_data){

    
    # data file paths
    simulated_data<-"qiime2_benchmark/otu_table1.txt"
    biom_file <- 'qiime2_benchmark/otu_table1.biom'
    output_qza <- 'qiime2_benchmark/otu_table1.qza'
    metadata<-"qiime2_benchmark/meta_table1.tsv"
    ancomoutput <- 'qiime2_benchmark/ancombc.qza'
    exported <- 'qiime2_benchmark/exported_ancompd'
    ppd_otu_filtered_qza="qiime2_benchmark/otu_pd_qiime_filtered.qza"
    meta.dat <- data.frame(X = raw_data$covariate, Z1 = raw_data$confounder[, 1],
                       Z2 = raw_data$confounder[, 2])
    otu.tab.sim <- raw_data$otu.tab.sim
    actual=ifelse(raw_data$diff.otu.ind, "DA", "NOT_DA")
    tr=data.frame("truth" = actual, row.names = rownames(otu.tab.sim), stringsAsFactors = F)
    meta.dat$X=ifelse(meta.dat$X, "healthy","unhealthy")
    #otu data processing for qiime2 input
    otu <- cbind(rownames(otu.tab.sim), otu.tab.sim[,2:length(colnames(otu.tab.sim))])
    colnames(otu)[1] <- "featureid"
    rownames(otu) <- NULL

    write.table(otu, simulated_data, sep = "\t", col.names=TRUE, row.names=FALSE, quote = FALSE)
    #metadata processing for qiime2 input
    colnames(meta.dat)=c("covariate", "cofounder1","cofounder2")

    metad  <- cbind(rownames(meta.dat),meta.dat)
    colnames(metad)[1] <- "sampleid"
    rownames(metad) <- NULL
    write.table(metad, metadata, sep = "\t", col.names=TRUE, row.names=FALSE, quote = FALSE)

   # convert data into biom format
    system(paste("biom convert",
                 "-i",simulated_data ,
                 "-o", biom_file ,
                 "--table-type=\"OTU table\"",
                 "--to-hdf5"), intern=TRUE)
    #import data into qiime2                
    system(paste("qiime tools import",
             "--input-path", biom_file,
             "--type 'FeatureTable[Frequency]'",
             "--input-format BIOMV210Format",
             "--output-path", output_qza), intern=TRUE)
    
    #ppd_otu_filtered_qza="~/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/data/PD/otu_pd_qiime_filtered.qza"
    system(paste("qiime feature-table filter-features",
             "--i-table", output_qza,
             "--p-min-frequency",50,
             "--p-min-samples",4,
             "--o-filtered-table", ppd_otu_filtered_qza), intern=TRUE)
                   
   # Run ANCOM-BC with sampled parameters
    system(paste("qiime composition ancombc",
             "--i-table",ppd_otu_filtered_qza,
             "--m-metadata-file", metadata,
             "--p-formula", "covariate",
             "--p-p-adj-method", "BH",
             "--p-neg-lb", FALSE,
             "--p-tol", 0.01,
             "--p-max-iter", 75,
             "--p-conserve", FALSE,
             "--o-differentials", ancomoutput))                
  
   # Export the results from QIIME2
   system(paste("qiime tools export", 
              "--input-path",ancomoutput,
             "--output-path",exported))                
  
  # Construct file path for q_val_slice.csv
   q_val_csv <- paste0(exported, "/q_val_slice.csv")
  
  # Check if the file exists before reading
  if (!file.exists(q_val_csv)) {
    cat("Expected q_val_slice.csv does not exist at path:", q_val_csv, "\n")
    next
  }
  
    # Read the q-value results
    ancom_q_val <- read.csv(file = q_val_csv, sep = ',', row.names = 1)

    # take only necessary data
    ancom_q_val=data.frame("covariateunhealthy" = ancom_q_val$covariateunhealthy, row.names = rownames(ancom_q_val), stringsAsFactors = F)
    #construct output dataframe
    e=!(rownames(otu.tab.sim)%in%rownames(ancom_q_val))
    nq=rownames(otu.tab.sim)[e]
    nqv=data.frame("covariateunhealthy" = rep(NA, length(nq)), row.names = nq, stringsAsFactors = F)
    q_val_all=rbind(ancom_q_val,nqv)
    names(q_val_all)[names(q_val_all) == "covariateunhealthy"] <- "q_value"
    pred= ifelse(ancom_q_val < 0.05, "DA", "NOT_DA")
    pred=as.data.frame(pred)
    d=!(rownames(otu.tab.sim)%in%rownames(pred))
    npred=rownames(otu.tab.sim)[d]
    ND=data.frame("covariateunhealthy" = rep("NOT_DA", length(npred)), row.names = npred, stringsAsFactors = F)
    pr=rbind(pred,ND)
    out_df=cbind(pr,q_val_all,tr)
    names(out_df)[names(out_df) == "covariateunhealthy"] <- "pred"

  return("method_out"=out_df)
}


In [9]:

# Load a specific simulated data

#simulation <- readRDS("data/simulated_data/sifm_otu.rds")
simulation <- readRDS("data/simulated_data/pd_otu.rds")
#simulation <- readRDS("data/simulated_data/ibd_otu.rds")
#simulation <- readRDS("data/simulated_data/ecam_otu.rds")
#simulation <- readRDS("data/simulated_data/american_gut_otu.rds")


#simulation <- readRDS("data/simulated_data/pd_otu_05.rds")

In [10]:
# specify corresponding sample size
#sample_size=list(length(rownames(list_of_dfs$ecam)),length(rownames(list_of_dfs$pd)))
sample_size=list(1000,2000,3000,4050,5100,6150,7200,8000,9000,10000) # for pd dataframe
#sample_size=list(1000,2000,3000,4000,5000,6000,7000,8000,9000,10000,11000,12000,13000) # for ecam dataframe
#sample_size=list(4000,5000,6000,6500,7000,7500,8000,8500,9000) # for IDB dataframe
#sample_size=list(1000,1100,1200,1400,1600,1700,1800,1900,2500) # for study_12021
#sample_size=list(1200,2500,3000,4500,5500,6500,7500,8400,9500,11000,12000,13000,14000,15000,16000,17000) # for for american_gut dataframe

In [11]:
#specify corresponding file path of original data
#ecam dataframe
#file_paths <- c("data/ecam_tutorial/ecam_otu.tsv")
#pd dataframe
file_paths <- c("data/PD/pd_otu.tsv")
#american_gut dataframe
#file_paths <- c("data/american_gut/american_gut_otu.tsv")
#IDB dataframe
#file_paths <- c("data/IBD/ibd_otu.tsv")
#Study_12021
#file_paths <- c("data/SIFM/sifm_otu.tsv")



list_of_dfs <- lapply(file_paths, function(file) read.table(file, sep = '\t', header=TRUE, row.names=1))

In [7]:
library(igraph)



Attaching package: ‘igraph’


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following object is masked from ‘package:base’:

    union




In [18]:
# Specify the method you want to run
methods_to_run <- c("ANCOM-BC")  # Add more if needed , "FlashWeave"  "ANCOM-BC"

# Initialize results_list
results_list <- list()
for(method in methods_to_run) {
  results_list[[method]] <- vector("list", length(list_of_dfs))
  for(i in 1:length(list_of_dfs)) {
    results_list[[method]][[i]] <- vector("list", length(sample_size))
  }
}

# Loop over datasets and methods
for(i in 1:length(list_of_dfs)) {
  for(j in 1:length(sample_size)) {
    data <- simulation[[i]][[j]]
    
    for(method in methods_to_run) {
      # Run the method
      run_id <- paste0("method_", method, "_dataset_", i, "_sample_", j)
      result <- run_method(data, method, run_id)
      
      # Store the result
      results_list[[method]][[i]][[j]] <- result
    }
  }
}


In [20]:
saveRDS(results_list, file = "pd_ancombc.rds")

In [28]:
E=results_list[[1]][[1]][[1]]$pred=="DA"

In [32]:
results_list[[1]][[1]][[1]]

,pred,q_value,truth
,<chr>,<dbl>,<chr>
TACAGAGGTCTCAAGCGTTGTTCGGAATCACTGGGCGTAAAGCGTGCGTAGGCTGTTTCGTAAGTCGTGTGTGAAAGGCGCGGGCTCAACCCGCGGACGGCACATGATACTGCGAGACTAGAGTAATGGAGGGGGAACCGGAATTCTCGG,NOT_DA,7.613667e-01,NOT_DA
TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAGCGTAGGCGGACGCTTAAGTCAGTTGTGAAAGTTTGCGGCTCAACCGTAAAATTGCAGTTGATACTGGGTGTCTTGAGTACAGTAGAGGCAGGCGGAATTCGTGG,NOT_DA,3.987529e-01,NOT_DA
TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAGCGTAGATGGATGTTTAAGTCAGTTGTGAAAGTTTGCGGCTCAACCGTAAAATTGCAGTTGATACTGGATATCTTGAGTGCAGTTGAGGCAGGCGGAATTCGTGG,NOT_DA,7.338081e-01,NOT_DA
TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAGCGTAGATGGATGTTTAAGTCAGTTGTGAAAGTTTGCGGCTCAACCGTAAAATTGCAGTTGATACTGGATGTCTTGAGTGCAGTTGAGGCAGGCGGAATTCGTGG,NOT_DA,5.528242e-01,DA
TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAGCGTAGGTGGACAGTTAAGTCAGTTGTGAAAGTTTGCGGCTCAACCGTAAAATTGCAGTTGATACTGGCTGTCTTGAGTACAGTAGAGGTGGGCGGAATTCGTGG,DA,3.894389e-18,DA
TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGTGCGTAGGCGGCCTTTTAAGTCAGCGGTGAAAGTCTGTGGCTCAACCATAGAATTGCCGTTGAAACTGGGGGGCTTGAGTATGTTTGAGGCAGGCGGAATGCGTGG,NOT_DA,3.252261e-01,NOT_DA
TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAGCGTAGACGGTTAAGCAAGTCTGAAGTGAAAGCCCGGGGCTCAACCCCGGTACTGCTTTGGAAACTGTTTGACTTGAGTGCAGGAGAGGTAAGTGGAATTCCTAG,DA,5.520201e-06,DA
TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAGCGTAGGTGGATTGTTAAGTCAGTTGTGAAAGTTTGCGGCTCAACCGTAAAATTGCAGTTGAAACTGGCAGTCTTGAGTACAGTAGAGGTGGGCGGAATTCGTGG,NOT_DA,9.587313e-01,NOT_DA
TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAGCGTAGGCGGAAGCTTAAGTCAGTTGTGAAAGTTTGCGGCTCAACCGTAAAATTGCAGTTGATACTGGGTTTCTTGAGTGCAGTAGAGGTAGGCGGAATTCGTGG,NOT_DA,9.587313e-01,NOT_DA


In [31]:
library(igraph)

# Iterate through the methods, datasets, and sample sizes
for (method in names(results_list)) {
  for (i in seq_along(results_list[[method]])) {
    for (j in seq_along(results_list[[method]][[i]])) {
      
      # Extract the graph
      graph <- results_list[[method]][[i]][[j]]
      
      # Define a unique file name
      file_name <- paste0("graph_", method, "_dataset", i, "_sample", j, ".graphml")
      
      # Save the graph to a GraphML file
      write_graph(graph, file = file_name, format = "graphml")
      print(paste("Saved graph to", file_name))
    }
  }
}


[1] "Saved graph to graph_FlashWeave_dataset1_sample1.graphml"
[1] "Saved graph to graph_FlashWeave_dataset1_sample2.graphml"
[1] "Saved graph to graph_FlashWeave_dataset1_sample3.graphml"
[1] "Saved graph to graph_FlashWeave_dataset1_sample4.graphml"
[1] "Saved graph to graph_FlashWeave_dataset1_sample5.graphml"
[1] "Saved graph to graph_FlashWeave_dataset1_sample6.graphml"
[1] "Saved graph to graph_FlashWeave_dataset1_sample7.graphml"
[1] "Saved graph to graph_FlashWeave_dataset1_sample8.graphml"
[1] "Saved graph to graph_FlashWeave_dataset1_sample9.graphml"
[1] "Saved graph to graph_FlashWeave_dataset1_sample10.graphml"


In [32]:
for (method in names(results_list)) {
  for (i in seq_along(results_list[[method]])) {
    for (j in seq_along(results_list[[method]][[i]])) {
      
      # Extract the graph
      graph <- results_list[[method]][[i]][[j]]
      
      # Ensure node names are preserved in the `name` attribute
      if (is.null(V(graph)$name)) {
        V(graph)$name <- as.character(V(graph))  # Assign default names if missing
      }
      
      # Save the graph to GraphML
      file_name <- paste0("graph_", method, "_dataset", i, "_sample", j, ".graphml")
      write_graph(graph, file = file_name, format = "graphml")
    }
  }
}


In [30]:
results_list

$FlashWeave
$FlashWeave[[1]]
$FlashWeave[[1]][[1]]
IGRAPH 1188f3b U-W- 564 522 -- 
+ attr: Feature (v/c), mv (v/n), id (v/c), weight (e/n)
+ edges from 1188f3b:
 [1] 1--  8 1-- 14 1-- 19 1-- 21 1-- 27 1-- 30 1-- 38 1-- 56 1-- 93 1--141
[11] 1--237 1--240 1--361 2--  3 2--  5 2--  8 2-- 10 2-- 15 2-- 16 2-- 22
[21] 2-- 23 2-- 24 2-- 25 2-- 27 2-- 28 2-- 31 2-- 37 2-- 41 2-- 44 2-- 50
[31] 2-- 55 2-- 58 2-- 66 2-- 95 2-- 96 2-- 97 2--109 2--112 2--119 2--144
[41] 2--158 2--240 2--318 3--  4 3--  8 3-- 22 3-- 25 3-- 28 3-- 31 3-- 39
[51] 3-- 50 3-- 54 3-- 97 3--102 3--112 3--126 3--144 3--158 3--294 4-- 13
[61] 4-- 20 4-- 39 4-- 42 4-- 87 5--  7 5-- 13 5-- 21 5-- 24 5-- 28 5-- 60
[71] 5-- 71 5-- 86 5-- 92 5-- 93 5-- 94 5-- 96 5--112 5--144 5--169 5--232
+ ... omitted several edges

$FlashWeave[[1]][[2]]
IGRAPH ee0362e U-W- 561 502 -- 
+ attr: Feature (v/c), mv (v/n), id (v/c), weight (e/n)
+ edges from ee0362e:
 [1] 1--  3 1--  8 1-- 28 1-- 30 1-- 46 1-- 57 1-- 88 1-- 95 1-- 96 1--160
[11

In [33]:
library(igraph)

# Iterate through the methods, datasets, and sample sizes
for (method in names(results_list)) {
  for (i in seq_along(results_list[[method]])) {
    for (j in seq_along(results_list[[method]][[i]])) {
      
      # Extract the graph
      graph <- results_list[[method]][[i]][[j]]
      
      # Define a unique file name
      file_name <- paste0("graph_", method, "_dataset", i, "_sample", j, ".csv")
      
      # Save the graph to a GraphML file
      write.csv(graph, file_name, row.names = TRUE)
      print(paste("Saved result to", file_name))
    }
  }
}



Attaching package: ‘igraph’


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following object is masked from ‘package:base’:

    union




[1] "Saved result to graph_ANCOM-BC_dataset1_sample1.csv"
[1] "Saved result to graph_ANCOM-BC_dataset1_sample2.csv"
[1] "Saved result to graph_ANCOM-BC_dataset1_sample3.csv"
[1] "Saved result to graph_ANCOM-BC_dataset1_sample4.csv"
[1] "Saved result to graph_ANCOM-BC_dataset1_sample5.csv"
[1] "Saved result to graph_ANCOM-BC_dataset1_sample6.csv"
[1] "Saved result to graph_ANCOM-BC_dataset1_sample7.csv"
[1] "Saved result to graph_ANCOM-BC_dataset1_sample8.csv"
[1] "Saved result to graph_ANCOM-BC_dataset1_sample9.csv"
[1] "Saved result to graph_ANCOM-BC_dataset1_sample10.csv"
